### 전체 지하철역 데이터 전처리(9호선은 없는듯 함)
2호선 순환라인에 대한 데이터만 사용예정

In [212]:
import warnings
warnings.filterwarnings(action='ignore')
import scipy as sp
import scipy.stats
import statsmodels.api as sm
import sklearn as sk
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as plt
import glob
set(sorted([f.name for f in mpl.font_manager.fontManager.ttflist]))
mpl.rc('font', family='NanumGothic')
mpl.rc('axes', unicode_minus=False)

In [52]:
path ='C:/Code/regression'
filenames = glob.glob(path + "/*sub.csv")

In [53]:
filenames

['C:/Code/regression\\2016sub.csv',
 'C:/Code/regression\\2017sub.csv',
 'C:/Code/regression\\2018sub.csv',
 'C:/Code/regression\\2019sub.csv']

In [60]:
sub_2016=pd.read_csv(filenames[0],encoding='euc-kr',thousands = ',')
sub_2017=pd.read_csv(filenames[1],encoding='euc-kr',thousands = ',')
sub_2018=pd.read_csv(filenames[2],encoding='euc-kr',thousands = ',')
sub_2019=pd.read_csv(filenames[3],encoding='euc-kr',thousands = ',')

In [56]:
# sub_2016 일별 합계 추가
sub_2016['tot']=sub_2016.iloc[:,4:].sum(axis=1)

In [61]:
sub_2016.tail(3)

,날짜,역번호,역명,구분,5to6,6to7,7to8,8to9,9to10,10to11,...,16to17,17to18,18to19,19to20,20to21,21to22,22to23,23to24,0to1,total
201297,2016-12-31,2826,수진,하차,37,71,62,137,148,129,...,288,267,315,280,232,301,296,207,73,4095
201298,2016-12-31,2827,모란(8),승차,45,57,115,155,152,175,...,270,293,244,233,149,149,152,93,21,3797
201299,2016-12-31,2827,모란(8),하차,114,56,73,104,134,135,...,271,240,263,210,144,169,171,118,153,3273


In [62]:
sub_2017.tail(3)

,날짜,역번호,역명,구분,5to6,6to7,7to8,8to9,9to10,10to11,...,16to17,17to18,18to19,19to20,20to21,21to22,22to23,23to24,0to1,total
200747,2017-12-31,2826,수진,하차,8,68,76,88,150,206,...,235,245,269,305,205,264,217,134,200,4062
200748,2017-12-31,2827,모란,승차,29,36,74,87,149,175,...,254,204,245,182,167,192,179,138,26,3304
200749,2017-12-31,2827,모란,하차,5,55,36,50,81,84,...,189,184,273,203,167,189,171,124,1082,3628


In [63]:
sub_2017.columns

Index(['날짜', '역번호', '역명', '구분', '5to6', '6to7', '7to8', '8to9', '9to10',
       '10to11', '11to12', '12to13', '13to14', '14to15', '15to16', '16to17',
       '17to18', '18to19', '19to20', '20to21', '21to22', '22to23', '23to24',
       '0to1', 'total'],
      dtype='object')

In [15]:
# 호선, 구분(휴일 여부) 날림( 휴일의 경우 2017년에만 존재하는 column)
del sub_2017['호선']
del sub_2017['구분']

In [49]:
sub_2018.tail(3)

,날짜,역번호,역명,구분,05 ~ 06,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,...,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,00 ~ 01,합 계
200747,2018-12-31,2826,수진,하차,18,91,136,356,214,222,...,342,390,451,395,288,248,327,203,387,5605
200748,2018-12-31,2827,모란,승차,55,84,284,425,202,219,...,343,391,334,233,157,180,139,94,189,4885
200749,2018-12-31,2827,모란,하차,15,87,133,329,194,144,...,285,335,387,283,181,216,185,113,828,4595


In [47]:
del sub_2018['호선']

In [48]:
sub_2018.columns

Index(['날짜', '역번호', '역명', '구분', '05 ~ 06', '06 ~ 07', '07 ~ 08', '08 ~ 09',
       '09 ~ 10', '10 ~ 11', '11 ~ 12', '12 ~ 13', '13 ~ 14', '14 ~ 15',
       '15 ~ 16', '16 ~ 17', '17 ~ 18', '18 ~ 19', '19 ~ 20', '20 ~ 21',
       '21 ~ 22', '22 ~ 23', '23 ~ 24', '00 ~ 01', '합 계'],
      dtype='object')

In [51]:
del sub_2019['호선']

In [58]:
sub_2019.tail()

,날짜,역번호,역명,구분,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,...,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후,합 계
200799,2019-12-31,2825,신흥,하차,20,87,86,243,208,189,...,501,510,562,493,378,301,257,154,163,5794
200800,2019-12-31,2826,수진,승차,111,224,542,813,436,343,...,427,436,395,245,219,155,126,68,82,6228
200801,2019-12-31,2826,수진,하차,27,97,115,433,259,198,...,373,463,501,452,354,332,322,256,163,5996
200802,2019-12-31,2827,모란,승차,73,95,342,482,253,210,...,366,375,418,242,199,152,189,126,65,5151
200803,2019-12-31,2827,모란,하차,15,99,140,386,189,175,...,306,277,403,279,213,195,186,130,407,4436


In [60]:
col=['날짜','역번호','역명','구분']
for i in range(5,25):
    if i != 24:
        a=str(i)+'to'+str(i+1)
        col.append(a)
    else:
        a='0to1'
        col.append(a)
col.append('total')
len(col)

25

In [61]:
col

['날짜',
 '역번호',
 '역명',
 '구분',
 '5to6',
 '6to7',
 '7to8',
 '8to9',
 '9to10',
 '10to11',
 '11to12',
 '12to13',
 '13to14',
 '14to15',
 '15to16',
 '16to17',
 '17to18',
 '18to19',
 '19to20',
 '20to21',
 '21to22',
 '22to23',
 '23to24',
 '0to1',
 'total']

In [62]:
sub_2016.columns=col
sub_2017.columns=col
sub_2018.columns=col
sub_2019.columns=col

In [66]:
sub_2019.tail(3)

,날짜,역번호,역명,구분,5to6,6to7,7to8,8to9,9to10,10to11,...,16to17,17to18,18to19,19to20,20to21,21to22,22to23,23to24,0to1,total
200801,2019-12-31,2826,수진,하차,27,97,115,433,259,198,...,373,463,501,452,354,332,322,256,163,5996
200802,2019-12-31,2827,모란,승차,73,95,342,482,253,210,...,366,375,418,242,199,152,189,126,65,5151
200803,2019-12-31,2827,모란,하차,15,99,140,386,189,175,...,306,277,403,279,213,195,186,130,407,4436


In [67]:
sub_2016.to_csv('2016sub.csv',encoding='euc-kr',index=False)
sub_2017.to_csv('2017sub.csv',encoding='euc-kr',index=False)
sub_2018.to_csv('2018sub.csv',encoding='euc-kr',index=False)
sub_2019.to_csv('2019sub.csv',encoding='euc-kr',index=False)

In [64]:
sub_2016=pd.read_csv("2016sub.csv",encoding='euc-kr')
sub_2017=pd.read_csv("2017sub.csv",encoding='euc-kr')
sub_2018=pd.read_csv("2018sub.csv",encoding='euc-kr')
sub_2019=pd.read_csv("2019sub.csv",encoding='euc-kr')

In [65]:
sub_2016.tail()

,날짜,역번호,역명,구분,5to6,6to7,7to8,8to9,9to10,10to11,...,16to17,17to18,18to19,19to20,20to21,21to22,22to23,23to24,0to1,total
201295,2016-12-31,2825,신흥,하차,23,62,77,114,223,150,...,359,403,390,362,258,347,246,183,106,4832
201296,2016-12-31,2826,수진,승차,85,119,202,291,327,283,...,338,368,256,170,146,106,93,47,145,4376
201297,2016-12-31,2826,수진,하차,37,71,62,137,148,129,...,288,267,315,280,232,301,296,207,73,4095
201298,2016-12-31,2827,모란(8),승차,45,57,115,155,152,175,...,270,293,244,233,149,149,152,93,21,3797
201299,2016-12-31,2827,모란(8),하차,114,56,73,104,134,135,...,271,240,263,210,144,169,171,118,153,3273


In [68]:
sub_2016.iloc[:,4:-1]

,5to6,6to7,7to8,8to9,9to10,10to11,11to12,12to13,13to14,14to15,15to16,16to17,17to18,18to19,19to20,20to21,21to22,22to23,23to24,0to1
0,342,1637,1753,1856,2438,2425,2548,2718,2973,2921,3341,3108,2656,2134,1859,1394,1344,1056,612,222
1,469,339,584,1144,1723,2087,2983,3527,3386,2612,3639,3554,3200,3076,2519,2281,2398,2056,996,58
2,68,306,554,939,689,665,884,956,1084,1216,1033,1035,951,792,618,402,309,192,114,34
3,103,127,110,191,298,582,700,604,731,900,1069,1339,1458,1499,1383,1284,1198,675,375,9
4,702,341,234,296,335,419,733,865,991,1300,1772,2175,2580,2627,2302,2315,2365,2140,1117,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201295,23,62,77,114,223,150,201,253,378,359,338,359,403,390,362,258,347,246,183,106
201296,85,119,202,291,327,283,292,231,318,256,303,338,368,256,170,146,106,93,47,145
201297,37,71,62,137,148,129,151,221,305,359,216,288,267,315,280,232,301,296,207,73
201298,45,57,115,155,152,175,215,322,323,307,327,270,293,244,233,149,149,152,93,21


In [81]:
sub_2016['17to20']=sub_2016.iloc[:,16:19].sum(axis=1)
sub_2017['17to20']=sub_2017.iloc[:,16:19].sum(axis=1)
sub_2018['17to20']=sub_2018.iloc[:,16:19].sum(axis=1)
sub_2019['17to20']=sub_2019.iloc[:,16:19].sum(axis=1)

In [94]:
sub_2016.to_csv('2016sub.csv',encoding='euc-kr',index=False)
sub_2017.to_csv('2017sub.csv',encoding='euc-kr',index=False)
sub_2018.to_csv('2018sub.csv',encoding='euc-kr',index=False)
sub_2019.to_csv('2019sub.csv',encoding='euc-kr',index=False)

In [95]:
# 2019에는 새로 생긴역이 있어 데이터 개수가 다르나 2호선에는 변화가 없어 상관없음
sub_2019

,날짜,역번호,역명,구분,5to6,6to7,7to8,8to9,9to10,10to11,...,17to18,18to19,19to20,20to21,21to22,22to23,23to24,0to1,total,17to20
0,2019-01-01,150,서울역,승차,348,321,348,741,940,1401,...,3447,3277,2898,2590,3004,2234,970,38,39559,9622
1,2019-01-01,150,서울역,하차,222,821,808,961,1472,1737,...,2572,2060,1991,1683,1217,764,456,149,31279,6623
2,2019-01-01,151,시청,승차,87,98,143,206,280,395,...,1062,1146,1119,992,904,570,211,6,11836,3327
3,2019-01-01,151,시청,하차,48,237,323,571,577,648,...,746,606,374,311,268,177,144,57,10359,1726
4,2019-01-01,152,종각,승차,669,318,217,203,333,365,...,2129,2024,1806,1796,1780,1296,497,31,21042,5959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200799,2019-12-31,2825,신흥,하차,20,87,86,243,208,189,...,510,562,493,378,301,257,154,163,5794,1565
200800,2019-12-31,2826,수진,승차,111,224,542,813,436,343,...,436,395,245,219,155,126,68,82,6228,1076
200801,2019-12-31,2826,수진,하차,27,97,115,433,259,198,...,463,501,452,354,332,322,256,163,5996,1416
200802,2019-12-31,2827,모란,승차,73,95,342,482,253,210,...,375,418,242,199,152,189,126,65,5151,1035


In [97]:
sub_2016.iloc[:,[0,1,2,3,24,25]]

,날짜,역번호,역명,구분,total,17to20
0,2016-01-01,150,서울역(150),하차,39337,6649
1,2016-01-01,150,서울역(150),승차,42631,8795
2,2016-01-01,151,시청(151),하차,12841,2361
3,2016-01-01,151,시청(151),승차,14635,4340
4,2016-01-01,152,종각(152),승차,25660,7509
...,...,...,...,...,...,...
201295,2016-12-31,2825,신흥,하차,4832,1155
201296,2016-12-31,2826,수진,승차,4376,794
201297,2016-12-31,2826,수진,하차,4095,862
201298,2016-12-31,2827,모란(8),승차,3797,770


In [98]:
sub_2016=sub_2016.iloc[:,[0,1,2,3,24,25]]
sub_2017=sub_2017.iloc[:,[0,1,2,3,24,25]]
sub_2018=sub_2018.iloc[:,[0,1,2,3,24,25]]
sub_2019=sub_2019.iloc[:,[0,1,2,3,24,25]]

In [99]:
sub_2016.tail()

,날짜,역번호,역명,구분,total,17to20
201295,2016-12-31,2825,신흥,하차,4832,1155
201296,2016-12-31,2826,수진,승차,4376,794
201297,2016-12-31,2826,수진,하차,4095,862
201298,2016-12-31,2827,모란(8),승차,3797,770
201299,2016-12-31,2827,모란(8),하차,3273,713


In [100]:
sub=pd.concat([sub_2016,sub_2017,sub_2018,sub_2019])

In [101]:
sub

,날짜,역번호,역명,구분,total,17to20
0,2016-01-01,150,서울역(150),하차,39337,6649
1,2016-01-01,150,서울역(150),승차,42631,8795
2,2016-01-01,151,시청(151),하차,12841,2361
3,2016-01-01,151,시청(151),승차,14635,4340
4,2016-01-01,152,종각(152),승차,25660,7509
...,...,...,...,...,...,...
200799,2019-12-31,2825,신흥,하차,5794,1565
200800,2019-12-31,2826,수진,승차,6228,1076
200801,2019-12-31,2826,수진,하차,5996,1416
200802,2019-12-31,2827,모란,승차,5151,1035


In [102]:
# 행 개수 확인
len(sub_2016)+len(sub_2017)+len(sub_2018)+len(sub_2019)

803604

In [103]:
# index 재할당
sub.reset_index(inplace=True,drop=True)

In [104]:
sub.tail()

,날짜,역번호,역명,구분,total,17to20
803599,2019-12-31,2825,신흥,하차,5794,1565
803600,2019-12-31,2826,수진,승차,6228,1076
803601,2019-12-31,2826,수진,하차,5996,1416
803602,2019-12-31,2827,모란,승차,5151,1035
803603,2019-12-31,2827,모란,하차,4436,959


In [105]:
sub.날짜=pd.to_datetime(sub['날짜'])

In [106]:
sub['year']=sub.날짜.apply(lambda x: x.year)
sub['month']=sub.날짜.apply(lambda x: x.month)

In [107]:
# 요일 찾기
import datetime
from datetime import date

t=['월','화','수','목','금','토','일']
sub["요일"]=sub['날짜'].apply(lambda x: t[x.weekday()])

In [108]:
sub[sub.역명=='시청']

,날짜,역번호,역명,구분,total,17to20,year,month,요일
201302,2017-01-01,151,시청,승차,10443,2883,2017,1,일
201303,2017-01-01,151,시청,하차,8296,1162,2017,1,일
201320,2017-01-01,201,시청,승차,7456,1901,2017,1,일
201321,2017-01-01,201,시청,하차,5136,958,2017,1,일
201852,2017-01-02,151,시청,승차,25222,11460,2017,1,월
...,...,...,...,...,...,...,...,...,...
802521,2019-12-30,201,시청,하차,29365,3103,2019,12,월
803054,2019-12-31,151,시청,승차,30154,10880,2019,12,화
803055,2019-12-31,151,시청,하차,30425,3391,2019,12,화
803072,2019-12-31,201,시청,승차,30938,12668,2019,12,화


In [109]:
sub.dtypes

날짜        datetime64[ns]
역번호                int64
역명                object
구분                object
total              int64
17to20             int64
year               int64
month              int64
요일                object
dtype: object

sub=sub[sub["구분"]=='승차']

### 2호선 퇴근시간만

In [110]:
# 2호선 퇴근시간만
sub2=sub[(200<sub["역번호"]) & (sub["역번호"]<300)].reset_index(drop=True)
# sub2.drop(['7to8', '8to9', '9to10'],1,inplace=True)

In [111]:
sub2

,날짜,역번호,역명,구분,total,17to20,year,month,요일
0,2016-01-01,201,시청(201),승차,9398,2752,2016,1,금
1,2016-01-01,201,시청(201),하차,7238,1540,2016,1,금
2,2016-01-01,202,을지로입구(202),하차,22895,4409,2016,1,금
3,2016-01-01,202,을지로입구(202),승차,24895,7366,2016,1,금
4,2016-01-01,203,을지로3가(203),승차,6406,1515,2016,1,금
...,...,...,...,...,...,...,...,...,...
146095,2019-12-31,248,양천구청,하차,7520,2593,2019,12,화
146096,2019-12-31,249,신정네거리,승차,10083,1388,2019,12,화
146097,2019-12-31,249,신정네거리,하차,10189,3346,2019,12,화
146098,2019-12-31,250,용두(동대문구청),승차,2674,667,2019,12,화


In [113]:
# 역명에서 역번호 제외
sub2["역명"]=sub2.역명.str.split("(",expand=True)[0]

In [114]:
sub2.tail(4)

,날짜,역번호,역명,구분,total,17to20,year,month,요일
146096,2019-12-31,249,신정네거리,승차,10083,1388,2019,12,화
146097,2019-12-31,249,신정네거리,하차,10189,3346,2019,12,화
146098,2019-12-31,250,용두,승차,2674,667,2019,12,화
146099,2019-12-31,250,용두,하차,2601,674,2019,12,화


In [115]:
sub2

,날짜,역번호,역명,구분,total,17to20,year,month,요일
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금
3,2016-01-01,202,을지로입구,승차,24895,7366,2016,1,금
4,2016-01-01,203,을지로3가,승차,6406,1515,2016,1,금
...,...,...,...,...,...,...,...,...,...
146095,2019-12-31,248,양천구청,하차,7520,2593,2019,12,화
146096,2019-12-31,249,신정네거리,승차,10083,1388,2019,12,화
146097,2019-12-31,249,신정네거리,하차,10189,3346,2019,12,화
146098,2019-12-31,250,용두,승차,2674,667,2019,12,화


In [116]:
len(sub2.역명.unique()),sub2.역명.unique()

(51,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '신천', '종합운동장',
        '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대', '서울대입구',
        '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청', '당산',
        '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동', '도림천',
        '양천구청', '신정네거리', '용두', '잠실새내'], dtype=object))

In [117]:
sub2[sub2.역명=='까치산']

,날짜,역번호,역명,구분,total,17to20,year,month,요일


In [118]:
# 2호선은 총 51개 역이 존재 역명에 신천이 잠실새내로 변경되어 두개의 이름 존재하고, 까치산은 데이터가 아예 없음 
sub2.loc[sub2['역명']=="신천",'역명']="잠실새내"

In [119]:
sub2

,날짜,역번호,역명,구분,total,17to20,year,month,요일
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금
3,2016-01-01,202,을지로입구,승차,24895,7366,2016,1,금
4,2016-01-01,203,을지로3가,승차,6406,1515,2016,1,금
...,...,...,...,...,...,...,...,...,...
146095,2019-12-31,248,양천구청,하차,7520,2593,2019,12,화
146096,2019-12-31,249,신정네거리,승차,10083,1388,2019,12,화
146097,2019-12-31,249,신정네거리,하차,10189,3346,2019,12,화
146098,2019-12-31,250,용두,승차,2674,667,2019,12,화


In [120]:
len(sub2.역명.unique()),sub2.역명.unique()

(50,
 array(['시청', '을지로입구', '을지로3가', '을지로4가', '동대문역사문화공원', '신당', '상왕십리', '왕십리',
        '한양대', '뚝섬', '성수', '건대입구', '구의', '강변', '잠실나루', '잠실', '잠실새내',
        '종합운동장', '삼성', '선릉', '역삼', '강남', '교대', '서초', '방배', '사당', '낙성대',
        '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '대림', '신도림', '문래', '영등포구청',
        '당산', '합정', '홍대입구', '신촌', '이대', '아현', '충정로', '용답', '신답', '신설동',
        '도림천', '양천구청', '신정네거리', '용두'], dtype=object))

In [121]:
np.sum(sub2.isnull())

날짜        0
역번호       0
역명        0
구분        0
total     0
17to20    0
year      0
month     0
요일        0
dtype: int64

### 신설동라인같은 역 제거

In [122]:
out=["까치산","신정네거리","양천구청","도림천","용답","신답","용두","신설동"]

In [123]:
sub2=sub2[~sub2.역명.isin(out)]

In [124]:
sub2.reset_index(inplace=True,drop=True)

In [125]:
sub2

,날짜,역번호,역명,구분,total,17to20,year,month,요일
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금
3,2016-01-01,202,을지로입구,승차,24895,7366,2016,1,금
4,2016-01-01,203,을지로3가,승차,6406,1515,2016,1,금
...,...,...,...,...,...,...,...,...,...
125641,2019-12-31,241,이대,하차,18833,3361,2019,12,화
125642,2019-12-31,242,아현,승차,11065,1998,2019,12,화
125643,2019-12-31,242,아현,하차,10524,2858,2019,12,화
125644,2019-12-31,243,충정로,승차,10987,3218,2019,12,화


### 2호선 데이터 공휴일 여부 표시

In [126]:
holi=pd.read_csv("holi.csv",encoding="euc-kr")

In [127]:
holi

,locdate,seq,isHoliday,dateName,year,week
0,20150101,1,Y,신정,2015,목
1,20150218,1,Y,설날,2015,수
2,20150219,1,Y,설날,2015,목
3,20150220,1,Y,설날,2015,금
4,20150301,1,Y,삼일절,2015,일
...,...,...,...,...,...,...
86,20190913,1,Y,추석,2019,금
87,20190914,1,Y,추석,2019,토
88,20191003,1,Y,개천절,2019,목
89,20191009,1,Y,한글날,2019,수


In [128]:
# seq 공휴일이 겹친경우 카운트 됨
holi[holi.seq==2].locdate

47    20171002
49    20171003
Name: locdate, dtype: int64

In [129]:
# 띠용 20171002는 seq 2 인데 한개 밖에 없눈데..
holi[(holi.locdate==20171002) | (holi.locdate==20171003)]

,locdate,seq,isHoliday,dateName,year,week
47,20171002,2,Y,임시공휴일,2017,월
48,20171003,1,Y,개천절,2017,화
49,20171003,2,Y,추석,2017,화


In [130]:
# isHoliday가 N인경우는 쉬는날이 아니었으므로 제거
holi=holi[holi.isHoliday!="N"]

In [131]:
holi.groupby("year").size()

year
2015    16
2016    17
2017    19
2018    18
2019    16
dtype: int64

In [132]:
holi.locdate=pd.to_datetime(holi.locdate,format="%Y%m%d")

In [133]:
# sub2.holi는 공휴일 -> 주말은 아님
sub2["holi"]=sub2["날짜"].isin(holi.locdate)

In [134]:
sub2[sub2["year"]==2017][sub2.holi==True]
# 1634개행이 아닌 1548개 --> 20171003이 개천절과 추석이 겹쳐있기 때문에 중복!

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi
31476,2017-01-01,201,시청,승차,7456,1901,2017,1,일,True
31477,2017-01-01,201,시청,하차,5136,958,2017,1,일,True
31478,2017-01-01,202,을지로입구,승차,24566,7001,2017,1,일,True
31479,2017-01-01,202,을지로입구,하차,22945,4080,2017,1,일,True
31480,2017-01-01,203,을지로3가,승차,8371,1772,2017,1,일,True
...,...,...,...,...,...,...,...,...,...,...
62345,2017-12-25,241,이대,하차,15629,3723,2017,12,월,True
62346,2017-12-25,242,아현,승차,6463,1093,2017,12,월,True
62347,2017-12-25,242,아현,하차,6218,1530,2017,12,월,True
62348,2017-12-25,243,충정로,승차,5253,898,2017,12,월,True


In [135]:
# 주말이거나 공휴일이면 holi가 True가 되도록 변경
sub2["holi"]=np.where((sub2["요일"].isin(["토","일"]))|(sub2["holi"]==True),True,False)

In [136]:
sub2[sub2.holi==True].head(60)

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금,True
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금,True
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금,True
3,2016-01-01,202,을지로입구,승차,24895,7366,2016,1,금,True
4,2016-01-01,203,을지로3가,승차,6406,1515,2016,1,금,True
5,2016-01-01,203,을지로3가,하차,5792,913,2016,1,금,True
6,2016-01-01,204,을지로4가,하차,3395,596,2016,1,금,True
7,2016-01-01,204,을지로4가,승차,3738,935,2016,1,금,True
8,2016-01-01,205,동대문역사문화공원,승차,11478,3067,2016,1,금,True
9,2016-01-01,205,동대문역사문화공원,하차,13282,3115,2016,1,금,True


#### 환승역 개수 매칭(2호선 외 환승 가능한 역 수)

In [137]:
tran=pd.read_csv("transfer.csv",encoding="euc-kr")

In [138]:
tran.head()

,역,환승 역 개수
0,왕십리,3
1,홍대입구,2
2,동대문역사문화공원,2
3,건대입구,1
4,당산,1


In [139]:
sub2=pd.merge(sub2, tran, how="left",left_on='역명', right_on="역")

In [140]:
del sub2["역"]

In [141]:
sub2.rename(columns={"환승 역 개수":"ctransfer"},inplace=True)

In [142]:
sub2.ctransfer=sub2.ctransfer.astype('int')

In [143]:
sub2[sub2.역명=='당산']

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi,ctransfer
72,2016-01-01,237,당산,승차,8418,1464,2016,1,금,True,1
73,2016-01-01,237,당산,하차,8932,1841,2016,1,금,True,1
158,2016-01-02,237,당산,하차,14171,3317,2016,1,토,True,1
159,2016-01-02,237,당산,승차,13105,3023,2016,1,토,True,1
244,2016-01-03,237,당산,승차,10603,2378,2016,1,일,True,1
...,...,...,...,...,...,...,...,...,...,...,...
125461,2019-12-29,237,당산,하차,11923,2768,2019,12,일,True,1
125546,2019-12-30,237,당산,승차,22233,5541,2019,12,월,False,1
125547,2019-12-30,237,당산,하차,26168,6759,2019,12,월,False,1
125632,2019-12-31,237,당산,승차,20596,4982,2019,12,화,False,1


In [144]:
sub2[sub2.ctransfer.isnull()]

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi,ctransfer


In [145]:
sub2

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi,ctransfer
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금,True,1
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금,True,1
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금,True,0
3,2016-01-01,202,을지로입구,승차,24895,7366,2016,1,금,True,0
4,2016-01-01,203,을지로3가,승차,6406,1515,2016,1,금,True,1
...,...,...,...,...,...,...,...,...,...,...,...
125641,2019-12-31,241,이대,하차,18833,3361,2019,12,화,False,0
125642,2019-12-31,242,아현,승차,11065,1998,2019,12,화,False,0
125643,2019-12-31,242,아현,하차,10524,2858,2019,12,화,False,0
125644,2019-12-31,243,충정로,승차,10987,3218,2019,12,화,False,1


In [146]:
# sub2.csv에 역의주소.xlsx merge (null데이터 확인도 같이)
sub_address = pd.read_excel('역주소.xlsx', enconding = 'euc-kr')

In [147]:
sub2.head(3)

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi,ctransfer
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금,True,1
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금,True,1
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금,True,0


In [148]:
sub_address.head(3)

,역명,구,동
0,시청,중구,서소문동
1,을지로입구,중구,을지로1가
2,을지로3가,중구,을지로3가


In [149]:
# 각 역명에 해당하는 구와 동을 merge해서 sub2.csv파일에 열 추가
sub2 = pd.merge(sub2, sub_address, how='left', on= '역명')

In [150]:
sub2.head(5)

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi,ctransfer,구,동
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금,True,1,중구,서소문동
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금,True,1,중구,서소문동
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금,True,0,중구,을지로1가
3,2016-01-01,202,을지로입구,승차,24895,7366,2016,1,금,True,0,중구,을지로1가
4,2016-01-01,203,을지로3가,승차,6406,1515,2016,1,금,True,1,중구,을지로3가


In [151]:
# 강남만 강남구가아닌 강남
sub2.구.unique()

array(['중구', '성동구', '광진구', '송파구', '강남', '서초구', '동작구', '관악구', '구로구',
       '영등포구', '마포구', '서대문구'], dtype=object)

In [152]:
sub2.loc[sub2.구=='강남','구']='강남구'

In [153]:
sub2.구.unique()

array(['중구', '성동구', '광진구', '송파구', '강남구', '서초구', '동작구', '관악구', '구로구',
       '영등포구', '마포구', '서대문구'], dtype=object)

In [154]:
sub2

,날짜,역번호,역명,구분,total,17to20,year,month,요일,holi,ctransfer,구,동
0,2016-01-01,201,시청,승차,9398,2752,2016,1,금,True,1,중구,서소문동
1,2016-01-01,201,시청,하차,7238,1540,2016,1,금,True,1,중구,서소문동
2,2016-01-01,202,을지로입구,하차,22895,4409,2016,1,금,True,0,중구,을지로1가
3,2016-01-01,202,을지로입구,승차,24895,7366,2016,1,금,True,0,중구,을지로1가
4,2016-01-01,203,을지로3가,승차,6406,1515,2016,1,금,True,1,중구,을지로3가
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125641,2019-12-31,241,이대,하차,18833,3361,2019,12,화,False,0,마포구,염리동
125642,2019-12-31,242,아현,승차,11065,1998,2019,12,화,False,0,마포구,아현동
125643,2019-12-31,242,아현,하차,10524,2858,2019,12,화,False,0,마포구,아현동
125644,2019-12-31,243,충정로,승차,10987,3218,2019,12,화,False,1,서대문구,충정로3가


sub2.to_csv("sub2.csv",index=False,encoding="euc-kr")

In [156]:
sub2.isnull().sum()

날짜           0
역번호          0
역명           0
구분           0
total        0
17to20       0
year         0
month        0
요일           0
holi         0
ctransfer    0
구            0
동            0
dtype: int64

```






```

### 기상 데이터 전처리_서울전체

In [405]:
tem_2019= pd.read_csv('2019day_1.csv',encoding='euc-kr')
tem_2018= pd.read_csv('2018day_1.csv',encoding='euc-kr')
tem_2017= pd.read_csv('2017day_1.csv',encoding='euc-kr')
tem_2016= pd.read_csv('2016day_1.csv',encoding='euc-kr')

In [406]:
tem_2019.columns[36]

'합계 일사량(MJ/m2)'

In [407]:
tem_2019.rename(columns={'합계 일사량(MJ/m2)':'합계 일사(MJ/m2)','합계 일조시간(hr)':'합계 일조 시간(hr)'},inplace=True)

In [408]:
# 2016년이 윤년이라 하루 더 있음
tem = pd.concat([tem_2019,tem_2018,tem_2017,tem_2016])

In [409]:
len(tem)

1461

In [410]:
set(tem.columns)-set(tem_2019.columns)

set()

In [411]:
tem.reset_index(drop=True,inplace=True)

In [412]:
np.where(tem.columns.isin(['일시','평균기온(°C)','평균 상대습도(%)','일강수량(mm)','합계 일사(MJ/m2)']))

(array([ 1,  2, 12, 24, 36], dtype=int64),)

In [413]:
tem=tem.iloc[:,np.where(tem.columns.isin(['일시','평균기온(°C)','평균 상대습도(%)','일강수량(mm)','합계 일사(MJ/m2)']))[0]]

In [418]:
tem.tail()

,date,avgtemp,rainfall,avghimidity,solar
1456,2016-12-27,-2.3,0.5,54.6,9.29
1457,2016-12-28,-3.7,NaN,49.0,9.22
1458,2016-12-29,-4.2,0.3,53.1,9.67
1459,2016-12-30,-3.9,NaN,50.9,6.79
1460,2016-12-31,0.7,0.0,65.4,5.28


In [416]:
tem.columns=["date","avgtemp","rainfall","avghimidity","solar"]

In [419]:
tem[tem.solar.isnull()]

,date,avgtemp,rainfall,avghimidity,solar
239,2019-08-28,26.1,NaN,66.2,NaN
240,2019-08-29,23.4,36.9,77.1,NaN
382,2018-01-18,2.1,0.0,59.6,NaN
390,2018-01-26,-14.8,NaN,34.9,NaN
410,2018-02-15,-0.7,NaN,28.0,NaN
835,2017-04-16,18.7,NaN,52.9,NaN
844,2017-04-25,16.0,NaN,56.8,NaN
1038,2017-11-05,8.4,NaN,52.5,NaN
1053,2017-11-20,-0.4,0.0,55.3,NaN
1060,2017-11-27,2.5,NaN,53.0,NaN


In [420]:
tem.date=pd.to_datetime(tem['date'])
tem['year']=tem.date.apply(lambda x: x.year)
tem['month']=tem.date.apply(lambda x: x.month)

In [421]:
msolar=tem.groupby(['year','month']).mean().reset_index()

In [422]:
a=np.where(tem.solar.isnull())[0]

In [423]:
for i in a:
    tem.solar[i]=msolar.solar[(msolar.year==tem.year[i]) & (msolar.month==tem.month[i])]

In [424]:
tem.iloc[a,:]

,date,avgtemp,rainfall,avghimidity,solar,year,month
239,2019-08-28,26.1,NaN,66.2,16.766897,2019,8
240,2019-08-29,23.4,36.9,77.1,16.766897,2019,8
382,2018-01-18,2.1,0.0,59.6,5.482414,2018,1
390,2018-01-26,-14.8,NaN,34.9,5.482414,2018,1
410,2018-02-15,-0.7,NaN,28.0,11.454815,2018,2
835,2017-04-16,18.7,NaN,52.9,17.461071,2017,4
844,2017-04-25,16.0,NaN,56.8,17.461071,2017,4
1038,2017-11-05,8.4,NaN,52.5,5.590000,2017,11
1053,2017-11-20,-0.4,0.0,55.3,5.590000,2017,11
1060,2017-11-27,2.5,NaN,53.0,5.590000,2017,11


In [426]:
tem.rainfall=tem['rainfall'].fillna(0)

In [427]:
tem.tail(5)

,date,avgtemp,rainfall,avghimidity,solar,year,month
1456,2016-12-27,-2.3,0.5,54.6,9.29,2016,12
1457,2016-12-28,-3.7,0.0,49.0,9.22,2016,12
1458,2016-12-29,-4.2,0.3,53.1,9.67,2016,12
1459,2016-12-30,-3.9,0.0,50.9,6.79,2016,12
1460,2016-12-31,0.7,0.0,65.4,5.28,2016,12


tem.to_csv("tem.csv",encoding='euc-kr',index=False)

In [1001]:
tem=pd.read_csv('tem.csv',encoding='euc-kr')

In [1003]:
tem.isnull().sum()

date           0
avgtemp        0
rainfall       0
avghimidity    0
solar          0
year           0
month          0
dtype: int64

```




```

## 대기질

In [228]:
from bs4 import BeautifulSoup

In [229]:
from urllib import request
from urllib.request import urlopen
from urllib.parse import urlencode, quote_plus
from urllib.parse import unquote

In [230]:
from datetime import datetime
from datetime import timedelta

In [231]:
date=[]
start = datetime(2016, 1, 1)
for _ in range(1461):
    date.append(start.strftime("%Y%m%d"))
    start=start+timedelta(days=1)

In [233]:
data=[]
for j in date:
    url='http://openAPI.seoul.go.kr:8088/KEY/xml/DailyAverageCityAir/1/100/'+j
    req=requests.get(url)
    html=req.text
    soup=BeautifulSoup(html, 'html.parser')
    data.append(soup.find_all('row'))

In [234]:
data=sum(data,[])

In [235]:
msrdt_del=[]
msrrgn_nml=[]
msrste_nml=[]
no2l=[]
o3l=[]
col=[]
so2l=[]
pm10l=[]
pm25l=[]

In [236]:
len(data)

34794

In [237]:
data[0:2]

[<row>
 <msrdt_de>20160101</msrdt_de>
 <msrrgn_nm>도심권</msrrgn_nm>
 <msrste_nm>중구</msrste_nm>
 <pm10>61</pm10>
 <pm25>35</pm25>
 <o3>0.005</o3>
 <no2>0.065</no2>
 <co>0.8</co>
 <so2>0.007</so2>
 </row>,
 <row>
 <msrdt_de>20160101</msrdt_de>
 <msrrgn_nm>도심권</msrrgn_nm>
 <msrste_nm>종로구</msrste_nm>
 <pm10>68</pm10>
 <pm25>51</pm25>
 <o3>0.005</o3>
 <no2>0.047</no2>
 <co>0.7</co>
 <so2>0.006</so2>
 </row>]

In [238]:
for i in range(len(data)):
    msrdt_del.append(data[i].find('msrdt_de').text)
    msrrgn_nml.append(data[i].find('msrrgn_nm').text)
    msrste_nml.append(data[i].find('msrste_nm').text)
    no2l.append(data[i].find('no2').text)
    o3l.append(data[i].find('o3').text)
    col.append(data[i].find('co').text)
    so2l.append(data[i].find('so2').text)
    pm10l.append(data[i].find('pm10').text)
    pm25l.append(data[i].find('pm25').text)

In [239]:
air=pd.DataFrame()
air['msrdt']=msrdt_del
air['msrrgn_nm']=msrrgn_nml
air['msrste_nm']=msrste_nml
air['no2']=no2l
air['o3']=o3l
air['co']=col
air['so2']=so2l
air['pm10']=pm10l
air['pm25']=pm25l

In [241]:
# 누락된 날짜
null_date=set(date)-set(air.msrdt.unique())

In [242]:
len(null_date)

69

In [243]:
data2=[]
for i in null_date:
    url='http://openAPI.seoul.go.kr:8088/KEY/xml/TimeAverageAirQuality/1/1000/'+ i
    req=requests.get(url)
    html=req.text
    soup=BeautifulSoup(html, 'html.parser')
    data2.append(soup.find_all('row'))

In [244]:
len(data2)

69

In [245]:
data2=sum(data2,[])

In [246]:
msrdt_del=[]
msrrgn_nml=[]
msrste_nml=[]
no2l=[]
o3l=[]
col=[]
so2l=[]
pm10l=[]
pm25l=[]

In [247]:
len(data2)

41025

In [248]:
for i in range(len(data2)):
    msrdt_del.append(data2[i].find('msrdt').text)
    msrste_nml.append(data2[i].find('msrste_nm').text)
    no2l.append(data2[i].find('no2').text)
    o3l.append(data2[i].find('o3').text)
    col.append(data2[i].find('co').text)
    so2l.append(data2[i].find('so2').text)
    pm10l.append(data2[i].find('pm10').text)
    pm25l.append(data2[i].find('pm25').text)

In [249]:
air_miss=pd.DataFrame()
air_miss['msrdt']=msrdt_del
air_miss['msrste_nm']=msrste_nml
air_miss['no2']=no2l
air_miss['o3']=o3l
air_miss['co']=col
air_miss['so2']=so2l
air_miss['pm10']=pm10l
air_miss['pm25']=pm25l

In [250]:
air

,msrdt,msrrgn_nm,msrste_nm,no2,o3,co,so2,pm10,pm25
0,20160101,도심권,중구,0.065,0.005,0.8,0.007,61,35
1,20160101,도심권,종로구,0.047,0.005,0.7,0.006,68,51
2,20160101,도심권,용산구,0.046,0.005,0.7,0.005,59,46
3,20160101,서북권,은평구,0.039,0.01,0.7,0.007,58,35
4,20160101,서북권,서대문구,0.06,0.007,0.9,0.005,69,39
...,...,...,...,...,...,...,...,...,...
34789,20191231,서남권,양천구,0.017,0.019,0.4,0.004,29,17
34790,20191231,동남권,강남구,0.018,0.021,0.4,0.003,22,15
34791,20191231,동남권,서초구,0.019,0.028,0.2,0.003,28,13
34792,20191231,동남권,송파구,0.022,0.016,0.4,0.003,25,17


In [251]:
air_miss

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25
0,201806080700,강남구,0.013,0.021,0.3,0.004,,21
1,201806082100,강남구,0.045,0.015,0.5,0.005,43,32
2,201806080300,강남구,0.012,0.022,0.3,0.005,19,14
3,201806082200,강남구,0.04,0.013,0.5,0.005,44,34
4,201806082300,강남구,0.036,0.014,0.5,0.005,49,36
...,...,...,...,...,...,...,...,...
41020,201803310400,중랑구,0.028,0.013,0.7,0.005,55,37
41021,201803310300,중랑구,0.029,0.013,0.7,0.006,58,40
41022,201803310200,중랑구,0.036,0.007,0.8,0.006,58,42
41023,201803310100,중랑구,0.039,0.005,0.8,0.006,60,42


In [252]:
# 일평균으로 구하기 위해 시간 부분 자르기
air_miss.msrdt=air_miss.msrdt.apply(lambda a: a[:-4])

In [253]:
air_miss

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25
0,20180608,강남구,0.013,0.021,0.3,0.004,,21
1,20180608,강남구,0.045,0.015,0.5,0.005,43,32
2,20180608,강남구,0.012,0.022,0.3,0.005,19,14
3,20180608,강남구,0.04,0.013,0.5,0.005,44,34
4,20180608,강남구,0.036,0.014,0.5,0.005,49,36
...,...,...,...,...,...,...,...,...
41020,20180331,중랑구,0.028,0.013,0.7,0.005,55,37
41021,20180331,중랑구,0.029,0.013,0.7,0.006,58,40
41022,20180331,중랑구,0.036,0.007,0.8,0.006,58,42
41023,20180331,중랑구,0.039,0.005,0.8,0.006,60,42


In [254]:
air_miss[['no2','o3','co','so2','pm10','pm25']]=air_miss.iloc[:,2:].apply(lambda a: pd.to_numeric(a))

In [256]:
del air['msrrgn_nm']

In [261]:
air_day=air_miss.groupby(['msrdt','msrste_nm']).mean().reset_index()

In [262]:
# air_day = 시간별 평균으로 구한 일 평균 대기질
air_day.isnull().sum()

msrdt         0
msrste_nm     0
no2          22
o3           24
co           27
so2          24
pm10         11
pm25          9
dtype: int64

In [264]:
air_plus_miss=pd.concat([air,air_day])

In [2]:
# 측정소별 권역코드
code=pd.read_csv('code.csv',encoding='utf-8')

In [266]:
code.tail(3)

,권역코드,측정소명
22,104,강동구
23,104,서초구
24,104,송파구


In [268]:
air_day=pd.merge(air_plus_miss,code,how='left',left_on='msrste_nm',right_on='측정소명').iloc[:,:-1]

In [274]:
air_day[['no2','o3','co','so2','pm10','pm25']]=air_day.iloc[:,2:-1].apply(lambda a: round(pd.to_numeric(a),3))

In [277]:
air_day

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25,권역코드
0,20160101,중구,0.065,0.005,0.800,0.007,61.000,35.000,100
1,20160101,종로구,0.047,0.005,0.700,0.006,68.000,51.000,100
2,20160101,용산구,0.046,0.005,0.700,0.005,59.000,46.000,100
3,20160101,은평구,0.039,0.010,0.700,0.007,58.000,35.000,101
4,20160101,서대문구,0.060,0.007,0.900,0.005,69.000,39.000,101
...,...,...,...,...,...,...,...,...,...
36514,20190908,용산구,0.020,0.031,0.446,0.004,29.833,22.667,100
36515,20190908,은평구,0.014,0.039,0.604,0.004,39.292,25.458,101
36516,20190908,종로구,0.015,0.030,0.454,0.003,32.542,20.750,100
36517,20190908,중구,0.016,0.035,NaN,0.003,33.208,19.750,100


In [278]:
# air_code=일평균 대기질을 구하지 못한 구에 대한 대체값으로 사용할 권역별 평균 대기질
air_code=air_day.groupby(['msrdt','권역코드']).mean().reset_index()

In [279]:
air_code

,msrdt,권역코드,no2,o3,co,so2,pm10,pm25
0,20160101,100,0.052667,0.005000,0.733333,0.006000,62.666667,44.000000
1,20160101,101,0.047000,0.008333,0.866667,0.006000,62.000000,41.000000
2,20160101,102,0.049625,0.004250,1.062500,0.005875,72.125000,46.875000
3,20160101,103,0.044857,0.007714,1.014286,0.005571,66.857143,45.142857
4,20160101,104,0.043500,0.004500,0.900000,0.005500,72.500000,42.500000
...,...,...,...,...,...,...,...,...
7300,20191231,100,0.014000,0.021333,0.400000,0.002667,22.333333,17.333333
7301,20191231,101,0.013333,0.022000,0.433333,0.002333,23.000000,11.333333
7302,20191231,102,0.015750,0.023000,0.387500,0.002750,24.375000,13.875000
7303,20191231,103,0.016429,0.021143,0.357143,0.003143,25.285714,16.000000


In [280]:
air_day.isnull().sum()

msrdt         0
msrste_nm     0
no2          22
o3           24
co           27
so2          24
pm10         11
pm25          9
권역코드          0
dtype: int64

In [281]:
for j in air_day.columns:
    index=np.where(air_day[j].isnull())[0]
    for i in index:
        air_day[j][i]=air_code[(air_code.msrdt==air_day.msrdt[i]) & (air_code.권역코드==air_day.권역코드[i])][j]

C:\Users\shs40\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [286]:
air_day.isnull().sum()

msrdt        0
msrste_nm    0
no2          0
o3           0
co           0
so2          0
pm10         0
pm25         0
권역코드         0
dtype: int64

In [285]:
air_day

,msrdt,msrste_nm,no2,o3,co,so2,pm10,pm25,권역코드
0,20160101,중구,0.065,0.005,0.800,0.007,61.000,35.000,100
1,20160101,종로구,0.047,0.005,0.700,0.006,68.000,51.000,100
2,20160101,용산구,0.046,0.005,0.700,0.005,59.000,46.000,100
3,20160101,은평구,0.039,0.010,0.700,0.007,58.000,35.000,101
4,20160101,서대문구,0.060,0.007,0.900,0.005,69.000,39.000,101
...,...,...,...,...,...,...,...,...,...
36514,20190908,용산구,0.020,0.031,0.446,0.004,29.833,22.667,100
36515,20190908,은평구,0.014,0.039,0.604,0.004,39.292,25.458,101
36516,20190908,종로구,0.015,0.030,0.454,0.003,32.542,20.750,100
36517,20190908,중구,0.016,0.035,0.450,0.003,33.208,19.750,100


air_day.to_csv("air_avgday.csv",encoding='euc-kr',index=False)

```





```

## 따릉이

#### 파일 병합

In [519]:
bike_data=pd.DataFrame(pd.read_csv(filenames[4],encoding='euc-kr',thousands = ','))

In [524]:
filenames[4]

'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여이력 정보_2017년_2분기_1.csv'

In [510]:
path ='C:/Code/regression/따릉이'
filenames = glob.glob(path + "/서울특별시*.csv")

In [513]:
len(filenames)

38

In [306]:
bike1=pd.DataFrame()
bike2=pd.DataFrame()
bike3=pd.DataFrame()
bike4=pd.DataFrame()

In [307]:
a1=0
for i in range(0,11):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='euc-kr',thousands = ','))
    a1+=len(bike_data)
    print(filenames[i],i,a1)
    print(bike_data.columns[[1,2,9,10]])
    bike_data=bike_data.iloc[:,[1,2,9,10]]
    bike_data.columns=['대여일시','대여소번호','이용시간_분', '이용거리_M']
    bike1=pd.concat([bike1,bike_data])

C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2016년_1.csv 0 482990
Index(['대여일시', '대여소번호', '이용시간(분)', '이용거리(M)'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2016년_2.csv 1 1298291
Index(['대여일시', '대여소번호', '이용시간(분)', '이용거리(M)'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2016년_3.csv 2 1580480
Index(['대여일시', '대여소번호', '이용시간(분)', '이용거리(M)'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2017년_1.csv 3 1935211
Index(['대여일시', '대여소번호', '이용시간(분)', '이용거리(M)'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2017년_2분기_1.csv 4 2699365
Index([''대여일시'', ''대여대여소번호'', ''이용시간(분)'', ''이용거리(M)''], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2017년_2분기_2.csv 5 3231868
Index([''대여일시'', ''대여대여소번호'', ''이용시간(분)'', ''이용거리(M)''], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2017년_3분기_1.csv 6 4150779
Index([''대여일시'', ''대여대여소번호'', ''이용시간(분)'', ''이용거리(M)''], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2017년_3분기_2.csv 7 50

In [308]:
bike1

,대여일시,대여소번호,이용시간_분,이용거리_M
0,2016-01-01 0:12,112,24,4140
1,2016-01-01 0:22,113,24,2850
2,2016-01-01 0:23,211,7,1020
3,2016-01-01 0:24,113,22,2850
4,2016-01-01 0:35,335,19,2190
...,...,...,...,...
784729,'2018-03-31 23:59:46','2034',26,5010
784730,'2018-03-31 23:59:52','266',7,950
784731,'2018-03-31 23:59:54','926',39,3130
784732,'2018-03-31 23:59:56','503',4,590


In [309]:
a2=0
for i in range(11,15):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='euc-kr',thousands = ','))
    a2+=len(bike_data)
    print(filenames[i],i,a2)
    print(bike_data.columns[[1,2,9,10]])
    bike_data=bike_data.iloc[:,[1,2,9,10]]
    bike_data.columns=['대여일시','대여소번호','이용시간_분', '이용거리_M']
    bike2=pd.concat([bike2,bike_data])

C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2018년_2분기_1.csv 11 677878
Index([''대여일시'', ''대여대여소번호'', ''이용시간(분)'', ''이용거리(M)''], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2018년_2분기_2.csv 12 1629324
Index([''대여일시'', ''대여대여소번호'', ''이용시간(분)'', ''이용거리(M)''], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2018년_2분기_3.csv 13 2196183
Index([''대여일시'', ''대여대여소번호'', ''이용시간(분)'', ''이용거리(M)''], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여이력 정보_2018년_2분기_4.csv 14 2782487
Index([''대여일시'', ''대여대여소번호'', ''이용시간(분)'', ''이용거리(M)''], dtype='object')


In [310]:
bike2

,대여일시,대여소번호,이용시간_분,이용거리_M
0,'2018-04-01 00:00:01','266',7,1390
1,'2018-04-01 00:00:16','1946',44,9170
2,'2018-04-01 00:00:17','1449',42,4700
3,'2018-04-01 00:00:20','2034',32,5430
4,'2018-04-01 00:00:23','141',10,1960
...,...,...,...,...
586299,'2018-06-30 23:59:55','505',9,1230
586300,'2018-06-30 23:59:55','1328',17,3600
586301,'2018-06-30 23:59:57','2376',20,2590
586302,'2018-06-30 23:59:57','1251',57,6640


In [311]:
a3=0
for i in range(15,30):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='cp949',thousands = ','))
    a3+=len(bike_data)
    print(filenames[i],i,a3)
    print(bike_data.columns[[1,2,9,10]])
    bike_data=bike_data.iloc[:,[1,2,9,10]]
    bike_data.columns=['대여일시','대여소번호','이용시간_분', '이용거리_M']
    bike3=pd.concat([bike3,bike_data])

C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201812.csv 15 497958
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201901.csv 16 974716
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201902.csv 17 1444441
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201903.csv 18 2319687
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201904.csv 19 3747454
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201905.csv 20 5969254
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201906_1.csv 21 6769214
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201906_2.csv 22 7569165
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_

In [312]:
bike3

,대여일시,대여소번호,이용시간_분,이용거리_M
0,2018-12-01 0:00,1135,3,550.0
1,2018-12-01 0:01,1113,3,700.0
2,2018-12-01 0:01,3100,2,430.0
3,2018-12-01 0:00,1357,4,940.0
4,2018-12-01 0:01,3538,3,410.0
...,...,...,...,...
428693,2019-08-27 09:32:58,99999,2,0.0
428694,2019-08-27 11:29:56,99999,5,0.0
428695,2019-08-28 11:24:58,99999,2,0.0
428696,2019-08-28 12:59:58,99999,25,0.0


In [313]:
a4=0
for i in range(30,38):
    bike_data=pd.DataFrame(pd.read_csv(filenames[i],encoding='cp949',thousands = ','))
    a4+=len(bike_data)
    print(filenames[i],i,a4)
    print(bike_data.columns[[1,2,9,10]])
    bike_data=bike_data.iloc[:,[1,2,9,10]]
    bike_data.columns=['대여일시','대여소번호','이용시간_분', '이용거리_M']
    bike4=pd.concat([bike4,bike_data])

C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201909_1.csv 30 800000
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201909_2.csv 31 1600000
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201909_3.csv 32 2007589
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201910_1.csv 33 2807589
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201910_2.csv 34 3607588
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201910_3.csv 35 4348612
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201911_1.csv 36 5148612
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201911_2.csv 37 5827442
Index(['대여일시', '대여 대여소번호', '이용시간', '이용거리'], dtype='object')


In [330]:
bike4

,대여일시,대여소번호,이용시간_분,이용거리_M
0,2019-09-03 08:44:43,646,23,1250.0
1,2019-09-04 00:10:08,1372,8,1620.0
2,2019-09-04 00:41:36,1337,21,0.0
3,2019-09-04 08:48:48,529,15,1220.0
4,2019-09-05 08:46:52,646,7,1360.0
...,...,...,...,...
5827437,2019-11-26 08:27:07,2183,59,8520.0
5827438,2019-11-27 08:26:19,2141,59,8680.0
5827439,2019-11-28 08:11:43,2183,40,10570.0
5827440,2019-11-29 08:12:31,2183,64,10560.0


In [43]:
# xlsx파일 다운

In [315]:
path ='C:/Code/regression/따릉이'
filenames = glob.glob(path + "/서울특별시*.xlsx")

In [316]:
len(filenames)

8

In [317]:
filenames

['C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201807_01.xlsx',
 'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201807_02.xlsx',
 'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201808.xlsx',
 'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201809_1.xlsx',
 'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201809_2.xlsx',
 'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201810_01.xlsx',
 'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201810_02.xlsx',
 'C:/Code/regression/따릉이\\서울특별시 공공자전거 대여정보_201811.xlsx']

In [318]:
bike5=pd.DataFrame()

In [319]:
%%time
a5=0
for i in range(len(filenames)):
    bike_data=pd.DataFrame(pd.read_excel(filenames[i],sheet_name="Sheet1",encoding='euc-kr',thousands = ','))
    a5+=len(bike_data)
    print(filenames[i],i,a5)
    print(bike_data.columns[[1,2,9,10]])
    bike_data=bike_data.iloc[:,[1,2,9,10]]
    bike_data.columns=['대여일시','대여소번호','이용시간_분', '이용거리_M']
    bike5=pd.concat([bike5,bike_data])

Wall time: 983 µs
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201807_01.xlsx 0 1048575
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201807_02.xlsx 1 1089882
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201808.xlsx 2 2115975
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201809_1.xlsx 3 3164550
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201809_2.xlsx 4 3549617
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201810_01.xlsx 5 4598192
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201810_02.xlsx 6 4964657
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')
C:/Code/regression/따릉이\서울특별시 공공자전거 대여정보_201811.xlsx 7 5922910
Index(['대여일시', '대여소번호', '이용시간', '이용거리'], dtype='object')


In [320]:
bike5

,대여일시,대여소번호,이용시간_분,이용거리_M
0,2018-07-01 00:00:30,2226,3,570
1,2018-07-01 00:01:29,129,3,660
2,2018-07-01 00:02:19,1351,2,20
3,2018-07-01 00:00:43,954,4,680
4,2018-07-01 00:02:35,623,2,10
...,...,...,...,...
958248,2018-11-30 23:04:14,2266,230,3940
958249,2018-11-30 23:45:53,1308,184,1200
958250,2018-11-30 23:01:20,406,370,1650
958251,2018-11-30 23:25:14,2213,551,6410


In [342]:
bike1[bike1["대여소번호"].isnull()]

,대여일시,대여소번호,이용시간_분,이용거리_M


In [343]:
bike2[bike2["대여소번호"].isnull()]

,대여일시,대여소번호,이용시간_분,이용거리_M


In [344]:
bike3[bike3["대여소번호"].isnull()]

,대여일시,대여소번호,이용시간_분,이용거리_M


In [345]:
bike4[bike4["대여소번호"].isnull()]

,대여일시,대여소번호,이용시간_분,이용거리_M


In [427]:
bike5[bike5["대여소번호"].isnull()]

,대여일시,대여소번호,이용시간_분,이용거리_M


### bike전처리

In [327]:
bike1=pd.read_csv("bike1.csv",encoding='euc-kr',thousands = ',')
bike2=pd.read_csv("bike2.csv",encoding='euc-kr',thousands = ',')
bike3=pd.read_csv("bike3.csv",encoding='euc-kr',thousands = ',')
bike4=pd.read_csv("bike4.csv",encoding='euc-kr',thousands = ',')
bike5=pd.read_csv("bike5.csv",encoding='euc-kr',thousands = ',')

In [82]:
# 특수기호 제외

In [336]:
bike1["대여일시"]=bike1["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
bike2["대여일시"]=bike2["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
bike3["대여일시"]=bike3["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
bike4["대여일시"]=bike4["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
bike5["대여일시"]=bike5["대여일시"].str.replace(pat=r'[^\sA-Za-z0-9:-]', repl= r'', regex=True)
# bike5이미 타임 스탬프 형식

In [341]:
bike1["대여소번호"]=bike1["대여소번호"].str.replace(pat=r"[\s']", repl= r'', regex=True)
bike2["대여소번호"]=bike2["대여소번호"].str.replace(pat=r"[\s']", repl= r'', regex=True)
# bike3["대여소번호"]=bike3["대여소번호"].str.replace(pat=r"[\s']", repl= r'', regex=True)
# bike4["대여소번호"]=bike4["대여소번호"].str.replace(pat=r"[\s']", repl= r'', regex=True)
# bike5["대여소번호"]=bike5["대여소번호"].str.replace(pat=r"[\s']", repl= r'', regex=True)

In [96]:
# 대여일시 문자형을 datetime형태로

In [346]:
bike1['대여일시']=pd.to_datetime(bike1['대여일시'])
bike2['대여일시']=pd.to_datetime(bike2['대여일시'])
bike3['대여일시']=pd.to_datetime(bike3['대여일시'])
bike4['대여일시']=pd.to_datetime(bike4['대여일시'])
bike5['대여일시']=pd.to_datetime(bike5['대여일시'])

In [359]:
bike1=bike1[bike1.대여소번호!='중랑센터']

In [360]:
bike1

,대여일시,대여소번호,이용시간_분,이용거리_M
0,2016-01-01 00:12:00,112,24,4140
1,2016-01-01 00:22:00,113,24,2850
2,2016-01-01 00:23:00,211,7,1020
3,2016-01-01 00:24:00,113,22,2850
4,2016-01-01 00:35:00,335,19,2190
...,...,...,...,...
7341163,2018-03-31 23:59:46,2034,26,5010
7341164,2018-03-31 23:59:52,266,7,950
7341165,2018-03-31 23:59:54,926,39,3130
7341166,2018-03-31 23:59:56,503,4,590


In [369]:
bike1=bike1[bike1.대여소번호!='상암센터정비실']

In [388]:
bike1=bike1[bike1.대여소번호!='위트콤공장']

In [386]:
bike1=bike1[bike1.대여소번호!='중랑정비팀test1005']

In [391]:
bike1.대여소번호=bike1.대여소번호.astype('int')

In [398]:
bike2=bike2[~bike2.대여소번호.isin(['중랑정비팀test1005','위트콤','위트콤공장','상암센터정비실','중랑센터'])]

In [400]:
bike2.대여소번호=bike2.대여소번호.astype('int')

In [409]:
bike4.이용거리_M=bike4.이용거리_M.astype('int')

In [418]:
len(bike1)+len(bike2)+len(bike3)+len(bike4)+len(bike5)

34168664

#### len(bike)-len(bike[bike.이용거리_M==0]) =33425895 다합치니까(bike)돌아가질 않음..

#### 5개로 쪼개진 bike데이터셋들 이용거리 0인경우 제거하고 연도별로 다시 저장

In [478]:
bike1=bike1[bike1.이용거리_M!=0]

In [481]:
bike2=bike2[bike2.이용거리_M!=0]
bike3=bike3[bike3.이용거리_M!=0]
bike4=bike4[bike4.이용거리_M!=0]
bike5=bike5[bike5.이용거리_M!=0]

In [ ]:
bike_2016=bike1[bike1.대여일시.apply(lambda a:np.where(a.year==2016,True,False))]

In [ ]:
bike_2017=bike1[bike1.대여일시.apply(lambda a:np.where(a.year==2017,True,False))]

In [ ]:
bike_2018_1=bike1[bike1.대여일시.apply(lambda a:np.where(a.year==2018,True,False))]

In [ ]:
bike_2018_2=bike2[:]

In [ ]:
bike_2018_3=bike3[bike3.대여일시.apply(lambda a:np.where(a.year==2018,True,False))]

In [ ]:
bike_2018_5=bike5[:]

In [ ]:
bik_2019_3=bike3[bike3.대여일시.apply(lambda a:np.where(a.year==2019,True,False))]

In [485]:
bike_2019_4=bike4[:]

In [497]:
bike_2018=pd.concat([bike_2018_1,bike_2018_2,bike_2018_3,bike_2018_5])
bike_2019=pd.concat([bik_2019_3,bike_2019_4])

In [4]:
len(bike_2016)+len(bike_2017)+len(bike_2018)+len(bike_2019)

33425895

In [5]:
bike_2016['시속']=round(bike_2016.이용거리_M/1000,2)/round(bike_2016.이용시간_분/60,2)

In [6]:
bike_2017['시속']=round(bike_2017.이용거리_M/1000,2)/round(bike_2017.이용시간_분/60,2)
bike_2018['시속']=round(bike_2018.이용거리_M/1000,2)/round(bike_2018.이용시간_분/60,2)
bike_2019['시속']=round(bike_2019.이용거리_M/1000,2)/round(bike_2019.이용시간_분/60,2)

In [7]:
bike_2016.대여일시=pd.to_datetime(bike_2016['대여일시'])
bike_2017.대여일시=pd.to_datetime(bike_2017['대여일시'])
bike_2018.대여일시=pd.to_datetime(bike_2018['대여일시'])
bike_2019.대여일시=pd.to_datetime(bike_2019['대여일시'])

In [8]:
# 16,17은 0분 없음
bike_2016=bike_2016[bike_2016.이용시간_분!=0]
bike_2017=bike_2017[bike_2017.이용시간_분!=0]
bike_2018=bike_2018[bike_2018.이용시간_분!=0]
bike_2019=bike_2019[bike_2019.이용시간_분!=0]

bike_2016.to_csv("bike_2016.csv",index=False,encoding="euc-kr")
bike_2017.to_csv("bike_2017.csv",index=False,encoding="euc-kr")
bike_2018.to_csv("bike_2018.csv",index=False,encoding="euc-kr")
bike_2019.to_csv("bike_2019.csv",index=False,encoding="euc-kr")